In [1]:
import os
os.chdir("../")



In [2]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

from src.params.api_params import *
from src.data_gathering import APIRequest

In [3]:
api_request = APIRequest()
data = api_request.read_data()
data_transformed = api_request.transform_data(data)
data_filtered = api_request.filter_data(data_transformed)
data_filtered

,id,name,name_es,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,7d_sell_sold,1d_buy_sold,1d_sell_sold,lastUpdate
0,4,Assassin Transform Weapon,0,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-24T08:00:01.483Z
1,6,((208738)),((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
2,15,Abomination Hammer,Martillo de abominación,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
3,24,Sealed Package of Snowballs,Paquete sellado de bolas de nieve,Consumable,Basic,82868.0,88.0,153187.0,126.0,1110.0,1626.0,122.0,592.0,2024-08-09T17:08:13.828Z
4,33,PvP Heavy Helm of the Warrior,Yelmo pesado PvP del guerrero,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-11T18:00:01.806Z


In [5]:
data_transformed = api_request.transform_data(data)
data_filtered = api_request.filter_data(data_transformed)
data_filtered

,id,name,name_es,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,...,lastUpdate,daily,1d_sell_sold_avg,1d_lower_than_7d_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,total_profit_gold,ROI
0,4,Assassin Transform Weapon,0,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,...,2022-09-24T08:00:01.483Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
1,6,((208738)),((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,...,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
2,15,Abomination Hammer,Martillo de abominación,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,...,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
3,24,Sealed Package of Snowballs,Paquete sellado de bolas de nieve,Consumable,Basic,82868.0,88.0,153187.0,126.0,1110.0,...,2024-08-09T17:08:13.828Z,29,232,False,232.0,11,19.0,209.0,00G 2S 09C,0.217
4,33,PvP Heavy Helm of the Warrior,Yelmo pesado PvP del guerrero,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,...,2023-05-11T18:00:01.806Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67141,102182,Elonian Bazaar Shade,Sombra de bazar eloniano,Consumable,Basic,0.0,0.0,0.0,0.0,0.0,...,2024-07-16T19:00:05.426Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
67142,102183,Sandswept Pistol,Pistola arenosa,Weapon,Exotic,0.0,0.0,0.0,0.0,0.0,...,2024-07-30T19:00:04.532Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
67143,102184,Corrupted Hero Pistol,Pistola de héroe corrupto,Weapon,Exotic,0.0,0.0,0.0,0.0,0.0,...,2024-07-18T01:00:04.294Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000
67144,102185,Corrupted Hero Weapon Locker,0,Container,Exotic,0.0,0.0,0.0,0.0,0.0,...,2024-07-16T17:00:17.343Z,0,0,False,0.0,0,0.0,0.0,00G 00S 0C,0.000


In [6]:
# filtros
# el 1day sold menor que una septima parte de 7d sold
data_filtered = data_transformed[data_transformed['1d_lower_than_7d_avg'] == True]

# daily_mod > 0
data_filtered = data_filtered[data_filtered['daily_mod'] > DAILY_MIN]
# ROI > 10 y < 100
data_filtered = data_filtered[data_filtered['ROI'] > ROI_MIN]
data_filtered = data_filtered[data_filtered['ROI'] < ROI_MAX]

# profit > 10 platas (1000)
PROFIT_MIN
data_filtered = data_filtered[data_filtered['profit'] >= PROFIT_MIN]

# ordenacion

data_filtered = data_filtered.sort_values('ROI', ascending=False)

data_filtered

,id,name,name_es,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_buy_sold,...,lastUpdate,daily,1d_sell_sold_avg,1d_lower_than_7d_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,total_profit_gold,ROI
10002,12392,Bowl of Mushroom Risotto,Cuenco de risotto con champiñones,Consumable,Fine,3725.0,1198.0,2241.0,2724.0,402.0,...,2024-08-09T17:08:17.382Z,1,30,True,26.0,1,1117.0,1117.0,00G 11S 17C,0.9327
20762,28593,Gungnir,Gungnir,Weapon,Exotic,1132.0,4420.0,385.0,9987.0,411.0,...,2024-08-09T17:08:23.370Z,1,46,True,25.0,1,4069.0,4069.0,00G 40S 69C,0.9206
20745,28576,Dragonspine,Espina de dragón,Weapon,Exotic,1292.0,4435.0,223.0,9909.0,1039.0,...,2024-08-09T17:08:23.361Z,2,57,True,47.0,2,3988.0,7976.0,00G 79S 76C,0.8991
15740,23201,Berserker's Ring,Anillo de berserker,Trinket,Exotic,1162.0,3331.0,1374.0,7419.0,327.0,...,2024-08-09T17:08:19.944Z,3,81,True,71.0,3,2975.0,8925.0,00G 89S 25C,0.8932
20752,28583,Dragonfury,Furia de dragón,Weapon,Exotic,938.0,4824.0,313.0,10691.0,355.0,...,2024-08-09T17:08:23.365Z,1,51,True,22.0,1,4263.0,4263.0,00G 42S 63C,0.8838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51715,80728,Vial of Magic,Vial de magia,CraftingMaterial,Ascended,1725.0,9839.0,3466.0,12796.0,1945.0,...,2024-08-09T17:08:28.277Z,5,180,True,117.0,5,1038.0,5190.0,00G 51S 90C,0.1055
15741,23202,Berserker's Ring,Anillo de berserker,Trinket,Exotic,476.0,29151.0,683.0,37908.0,194.0,...,2024-08-09T17:08:19.946Z,1,61,True,38.0,1,3071.0,3071.0,00G 30S 71C,0.1053
7243,9567,18 Slot Craftsman's Bag,Saco de artesano de 18 casillas,Bag,Fine,735.0,21221.0,725.0,27590.0,117.0,...,2024-08-09T17:08:16.456Z,1,39,True,32.0,1,2230.0,2230.0,00G 22S 30C,0.1051
31536,44950,Superior Sigil of Malice,Sello superior de malicia,UpgradeComponent,Exotic,824.0,81310.0,213.0,105498.0,443.0,...,2024-08-09T17:08:25.500Z,3,72,True,70.0,3,8363.0,25089.0,02G 50S 89C,0.1029


In [30]:
def value_gold(value):

    str_value = str(int(value))

    gold = str_value[:-4] if str_value[:-4] != '' else '00'
    silver = str_value[-4:-2]
    copper = str_value[-2:]

    gold_value = f"{gold}G {silver}S {copper}C"

    return gold_value


In [31]:
value = 3988.0	

value_gold(value)

00


'00G 39S 88C'

In [17]:
value = str(25104)

value

gold = value[:-4]
silver = value[-4:-2]
copper = value[-2:]





gold, silver, copper

('2', '51', '04')

In [ ]:
columns_selected = [
    'id', 
    'name', 
    'type', 
    'rarity', 
    'buy_quantity', 
    'buy_price',
    'sell_quantity', 
    'sell_price', 
    '7d_buy_sold', 
    '7d_sell_sold',
    '1d_buy_sold', 
    '1d_sell_sold', 
    'lastUpdate', 
    # 'daily',
    # '1d_sell_sold_avg', 
    # '1d_lower_avg', 
    # '1d_sell_sold_mod', 
    'daily_mod',
    'profit', 
    'total_profit', 
    'ROI'
]

In [29]:
def calculate_roi(sell_price, buy_price):

    if buy_price != 0:
        roi = round((sell_price * 0.85 - buy_price) / buy_price, 4)
    else:
        roi = 0
    
    return roi

In [30]:
data_transformed = data.copy()
data_transformed['daily'] = data_transformed['1d_sell_sold'].apply(
    lambda x: math.floor(x * 0.05)
)
data_transformed['1d_sell_sold_avg'] = data_transformed['7d_sell_sold'].apply(
    lambda x: math.floor(x / 7) 
)
data_transformed['1d_lower_avg'] = (data_transformed['1d_sell_sold'] < data_transformed['1d_sell_sold_avg'])

data_transformed['1d_sell_sold_mod'] = data_transformed.apply(
    lambda row: min(row['1d_sell_sold'], row['1d_sell_sold_avg']),
    axis=1
)
data_transformed['daily_mod'] = data_transformed['1d_sell_sold_mod'].apply(
    lambda x: math.floor(x * 0.05)
)


data_transformed['profit'] = round(data_transformed['sell_price'] * 0.85 - data_transformed['buy_price'])
data_transformed['total_profit'] = round(data_transformed['profit'] * data_transformed['daily_mod'])
data_transformed['ROI'] = data_transformed.apply(
    lambda row: calculate_roi(row['sell_price'], row['buy_price']),
    axis=1
)


# (data_transformed['sell_price'] * 0.85 - data_transformed['buy_price'])/data_transformed['buy_price']


data_transformed.head()

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,...,1d_buy_sold,lastUpdate,daily,1d_sell_sold_avg,1d_lower_avg,1d_sell_sold_mod,daily_mod,profit,total_profit,ROI
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-09-24T08:00:01.483Z,0,0,False,0.0,0,0.0,0.0,0.0000
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2022-04-11T18:07:06.543Z,0,0,False,0.0,0,0.0,0.0,0.0000
3,24,Sealed Package of Snowballs,Consumable,Basic,82969.0,88.0,153151.0,127.0,2497.0,129.0,...,97.0,2024-08-09T10:20:13.740Z,27,356,False,356.0,17,20.0,340.0,0.2267
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2023-05-11T18:00:01.806Z,0,0,False,0.0,0,0.0,0.0,0.0000


In [52]:
field_string =f"fields={','.join(fields)}" 
field_string

'fields=id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate'

In [53]:
filter_string = f"filter={','.join(filters)}"
filter_string

'filter='

In [54]:
query_url = f"{DATA_GW2_URL}?{field_string}&{filter_string}"
# query_url = f"{DATA_GW2_URL}?{field_string}"
query_url

'https://api.datawars2.ie/gw2/v1/items/csv?fields=id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate&filter='

In [57]:
data = pd.read_csv(query_url).fillna(0)
data.head()

,id,name,type,rarity,buy_quantity,buy_price,sell_quantity,sell_price,7d_sell_sold,7d_sell_price_avg,1d_sell_sold,1d_sell_price_avg,7d_buy_sold,1d_buy_sold,lastUpdate
0,4,Assassin Transform Weapon,Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-09-24T08:00:01.483Z
1,6,((208738)),Weapon,Basic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
2,15,Abomination Hammer,Weapon,Fine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-04-11T18:07:06.543Z
3,24,Sealed Package of Snowballs,Consumable,Basic,83008.0,88.0,153150.0,127.0,2497.0,129.0,542.0,122.0,1025.0,97.0,2024-08-09T08:48:09.914Z
4,33,PvP Heavy Helm of the Warrior,Armor,Exotic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-11T18:00:01.806Z


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67146 entries, 0 to 67145
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67146 non-null  int64  
 1   name               67146 non-null  object 
 2   type               67146 non-null  object 
 3   rarity             67146 non-null  object 
 4   buy_quantity       67146 non-null  float64
 5   buy_price          67146 non-null  float64
 6   sell_quantity      67146 non-null  float64
 7   sell_price         67146 non-null  float64
 8   7d_sell_sold       67146 non-null  float64
 9   7d_sell_price_avg  67146 non-null  float64
 10  1d_sell_sold       67146 non-null  float64
 11  1d_sell_price_avg  67146 non-null  float64
 12  lastUpdate         67146 non-null  object 
dtypes: float64(8), int64(1), object(4)
memory usage: 6.7+ MB
